In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Data description
<ul>
    <li><b>id</b></b> - Unique ID for the customer.</li>
    <li><b>Gender</b> - Gender of the customer.</li>
    <li><b>Age</b> - Age of the customer.</li>
    <li><b>DrivingLicense</b> - 0: Customer does not have DL, 1: Customer already has DL RegionCode- Unique code for the region of the customer.</li>
    <li><b>Previously_Insured</b> - 1: Customer already has Vehicle Insurance, 0: Customer doesn't have Vehicle Insurance VehicleAge- Age of the Vehicle.</li> 
    <li><b>VehicleDamage</b> - 1: Customer got his/her vehicle damaged in the past, 0: Customer didn't get his/her vehicle damaged in the past.</li>
    <li><b>AnnualPremium</b> - The amount customer needs to pay as premium in the year.</li>
    <li><b>PolicySalesChannel</b> - Anonymised Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.</li> 
    <li><b>Vintage</b> - Number of Days, Customer has been associated with the company.</li> 
    <li><b>Response</b> - 1: Customer is interested, 0: Customer is not interested.</li> 
</ul>

### Task

Build a prediction model about the client's interest in the further use of the service.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission_iA3afxn.csv")

In [11]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [24]:
train.info()
test.info()
sample.info()

for df, df_names in (train, 'train'), (test, 'test'), (sample, 'sample'):
    print("\n\nThere is {} duplicated values in '{}' data frame".format(df.duplicated().sum(), df_names))
    print("There is {} missing values in '{}' data frame".format(df.isnull().sum().sum(), df_names))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127037 entries, 0 to 127036
Data columns (total 